In [1]:
# Imports
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
from datetime import datetime
import sqlite3

# --- 1. Database Setup ---
def setup_database():
    """Creates the database and table if they don't exist."""
    conn = sqlite3.connect('attendance.db')
    cursor = conn.cursor()
    # Create table: id, student_name, timestamp
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_name TEXT NOT NULL,
            timestamp TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

def log_attendance(student_name):
    """Logs a recognized student into the database with a timestamp."""
    conn = sqlite3.connect('attendance.db')
    cursor = conn.cursor()
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    # Insert a new record
    cursor.execute("INSERT INTO attendance_logs (student_name, timestamp) VALUES (?, ?)", (student_name, timestamp))
    conn.commit()
    conn.close()
    print(f"Logged attendance for {student_name} at {timestamp}")

# --- 2. Load Model and Labels ---
model_path = './model/face_recognition_model.h5'
labels_path = './model/label_names.npy'

if not os.path.exists(model_path) or not os.path.exists(labels_path):
    raise Exception("Model or labels file not found. Please run model_training.py first.")

model = load_model(model_path)
label_names = np.load(labels_path)

# --- 3. Initialize Attendance System ---
setup_database() # Ensure database is ready
todays_attendance = set() # To avoid duplicate logs in the same session
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def recognize_face(face_img):
    """Predicts the name for a given face image."""
    face_normalized = face_img / 255.0
    face_reshaped = np.reshape(face_normalized, (1, 100, 100, 1))
    prediction = model.predict(face_reshaped, verbose=0) # verbose=0 to hide predict output
    predicted_class_index = np.argmax(prediction)
    confidence = prediction[0][predicted_class_index]
    
    if confidence > 0.8: # Increased confidence for better accuracy
        return label_names[predicted_class_index]
    return None

# --- 4. Start Real-time Attendance Marking ---
cap = cv2.VideoCapture(0)
print("Starting attendance system. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (100, 100))
        recognized_name = recognize_face(face_resized)
        
        display_text, color = ("Unknown", (0, 0, 255))
        
        if recognized_name:
            display_text, color = (f"Recognized: {recognized_name}", (0, 255, 0))
            if recognized_name not in todays_attendance:
                log_attendance(recognized_name)
                todays_attendance.add(recognized_name)
            
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
    cv2.imshow('Attendance System', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

print("\nSession ended.")
print("Today's attendance marked for:", list(todays_attendance))

Starting attendance system. Press 'q' to quit.
Logged attendance for Shafqat Rasool at 2025-10-19 18:13:55

Session ended.
Today's attendance marked for: [np.str_('Shafqat Rasool')]
